# DDPM improvements

## Sortie de l'espace de définition

Problème notoire : les données générées ne sont pas contraintes de rester dans l'intervalle de définition d'un pixel, la preuve :

In [ ]:
from music_diffusion.networks import Denoiser, Noiser, normal_kl_div
from music_diffusion.data import OUTPUT_SIZES
import torch as th
from tqdm import tqdm

In [ ]:
input_channels = 2
T = 100
denoiser = Denoiser(input_channels, T, 16, 1e-4, 2e-2, [(8, 16), (16, 24), (24, 32)], 8)

betas = th.linspace(1e-4, 2e-2, steps=T)
alphas = 1 - betas
alphas_cum_prod = th.cumprod(alphas, dim=0)

In [ ]:
with th.no_grad():
    denoiser.cuda()
    denoiser.eval()

    # on tire une image aléatoire : input pour générer la donnée
    x_t = th.randn(1, input_channels, *OUTPUT_SIZES, device="cuda")

    # les étapes de T - 1 à 0
    for t in tqdm(reversed(range(T))):
        z = th.randn_like(x_t, device="cuda") if t > 0 else th.zeros_like(x_t, device="cuda")

        # predire le bruit
        eps_theta, _ = denoiser(
            x_t.unsqueeze(1),
            th.tensor([[t]], device="cuda"),
        )
        eps_theta = eps_theta.squeeze(1)

        # moyenne
        mu = (x_t - eps_theta * (1 - alphas[t]) / th.sqrt(1 - alphas_cum_prod[t])) / th.sqrt(alphas[t])

        # variance
        var = betas[t]

        # création du x à l'étape t - 1
        x_t = mu + th.sqrt(var) * z

In [ ]:
x_t.max(), x_t.min()

Solution : en fouillant internet je suis tombé sur cette issue : https://github.com/hojonathanho/diffusion/issues/5

Originellement la formule de génération présentée par les auteurs est :

![](https://user-images.githubusercontent.com/32613612/118956366-344b0080-b968-11eb-8872-61f7aded9db1.png)

Dans la pratique ils n'ont pas codé ça, mais plutôt ceci :

![](https://user-images.githubusercontent.com/32613612/118959112-ae7c8480-b96a-11eb-971c-930a6271f7e2.png)

Ce qui donne en code :


In [ ]:
alphas_cum_prod_prev = th.cat([th.tensor([1]), alphas_cum_prod[:-1]], dim=0)

In [ ]:
with th.no_grad():
    denoiser.cuda()
    denoiser.eval()

    # on tire une image aléatoire : input pour générer la donnée
    x_t = th.randn(1, input_channels, *OUTPUT_SIZES, device="cuda")

    # les étapes de T - 1 à 0
    for t in tqdm(reversed(range(T))):
        z = th.randn_like(x_t, device="cuda") if t > 0 else th.zeros_like(x_t, device="cuda")

        # predire le bruit
        eps_theta, _ = denoiser(
            x_t.unsqueeze(1),
            th.tensor([[t]], device="cuda"),
        )
        eps_theta = eps_theta.squeeze(1)

        # moyenne
        mu_recon = th.clip(x_t / th.sqrt(alphas_cum_prod[t]) - th.sqrt((1 - alphas_cum_prod[t]) / alphas_cum_prod[t]) * eps_theta, -1, 1)
        mu = betas[t] * th.sqrt(alphas_cum_prod_prev[t]) / (1 - alphas_cum_prod[t]) * mu_recon + x_t * (1 - alphas_cum_prod_prev[t]) * th.sqrt(alphas[t]) / (1 - alphas_cum_prod[t])

        # variance
        var = betas[t]

        # création du x à l'étape t - 1
        x_t = mu + th.sqrt(var) * z

Et nous restons bien dans l'intervalle [-1;1]!

L'explication de pourquoi c'est toujours juste théoriquement ? Aucune idée lol.

In [ ]:
x_t.min(), x_t.max()

D'après la issue GitHub, il s'agit d'un "hack" mais qui fonctionne !

## Autre amélioration : variance interpolation

Une autre amélioration (disponible sur le papier DDPM improvements) est de ne plus mettre une constante pour la variance à chaque étape, mais de laisser le choix à l'algorithme d'utiliser entre deux facteurs temporels (facteurs qui dépendent de l'étape de diffusion) :

![](./resources/v_interpolation.png)

avec "betas tiddle" :

![](./resources/beta_tiddle.png)

OK donc il faut rajouter une autre sortie pour note UNet, sortie qui devra être entre [0; 1] pour l'interpolation.

Le tout avec une sigmoid ! Les auteurs ne contraignent pas l'espace de sortie (couche sans activation), dans la pratique ça se transforme en NaN :,(




In [ ]:
print(denoiser)

Une étape de la loop de génération sera la suivante (seulement la partie pour la variance) :


In [ ]:
betas_tiddle = betas * (1 - alphas_cum_prod_prev) / (1 - alphas_cum_prod)

In [ ]:
with th.no_grad():
    t = 99

    denoiser.cpu()
    x_t = th.randn_like(x_t, device="cpu")
    z = th.randn_like(x_t, device="cpu")

    eps_theta, v_theta = denoiser(
        x_t.unsqueeze(1),
        th.tensor([[t]]),
    )
    eps_theta = eps_theta.squeeze(1)
    v_theta = v_theta.squeeze(1)

    # variance
    var = th.exp(v_theta * th.log(betas[t]) + (1 - v_theta) * th.log(betas_tiddle[t]))

In [ ]:
var.max(), var.min()

La génération déjà codée par mes soins :

In [ ]:
with th.no_grad():
    x_0 = denoiser.sample(th.randn(1, input_channels, *OUTPUT_SIZES), verbose=True)

In [ ]:
x_0.min(), x_0.max()

Ok mais comment on l'intègre dans la MSE du bruit versus le bruit prédit ?

=> Avec un nouvel objectif : la divergence de Kullback-Leibler (à additionner avec la MSE) :

$$KL(q(x_{t-1}|x_{t}, x_{0}) \| p_{\theta}(x_{t-1}|x_{t}))$$

Ok mais comment on calcule cette divergence de KL ?

=> Il faut calculer la moyenne et la variance pour nos deux distributions : celle du "noiser" et celle du "denoiser". De la même manière qu'on calculait la moyenne et la variance lors de la génération pour le denoiser, pour le noiser c'est dans l'article.

La divergence de KL se fera donc entre ces deux paires de moyennes et de variances :

![](./resources/normal_kl_div.png)

## Last pb

Ok c'est cool ces améliorations mais est-ce que ça change la donne ? Oui et non :
La donnée est bien dans l'espace de définition mais :
- la divergence de KL **explose** (milliard de milliards)
- Le modèle "boude" la magnitude (presque **aucun motifs musicaux**, du silence en gros)

Solution :
- sommer pour la divergence de KL sur les pixels du spectrogramme
- ajouter un facteur pour la divergence de KL (on divise par le max selon le batch) pour éviter qu'elle n'explose
- enlever la MSE qui "interfère" avec la divergence de KL

In [ ]:
noiser = Noiser(T, 1e-4, 2e-2)

In [ ]:
# final train loop body

batch_size = 1
step_batch_size = 2

t = th.randint(0, T, (batch_size, step_batch_size,))

x_t, _ = noiser(x_0, t)
eps_theta, v_theta = denoiser(x_t, t)

# Avant
# loss = mse(eps, eps_theta)
# loss = loss.mean()

# Maintenant
q_mu, q_var = noiser.posterior(x_t, x_0, t)
p_mu, p_var = denoiser.prior(x_t, t, eps_theta, v_theta)

print(q_mu.size(), q_var.size())
print(p_mu.size(), p_var.size())

loss = normal_kl_div(q_mu, q_var, p_mu, p_var)
print(loss.size(), loss.max())

loss = loss / loss.detach().max()
print(loss.size(), loss.max())

loss = loss.mean()
print(loss.size(), loss.max())

Let's go générer de la musique !